In [147]:
import requests
import os
import pandas as pd

TOKEN_URL = 'https://accounts.spotify.com/api/token'
BASE_URL = 'https://api.spotify.com/v1/'

# 1. Retrieve access token and call Spotify API

In [148]:
os.environ

environ{'USER': 'lucie_unix',
        'VSCODE_WSL_EXT_LOCATION': '/mnt/c/Users/Lucie/.vscode/extensions/ms-vscode-remote.remote-wsl-0.88.5',
        'SHLVL': '0',
        'HOME': '/home/lucie_unix',
        'OLDPWD': '/mnt/c/Users/Lucie/AppData/Local/Programs/Microsoft VS Code',
        'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus',
        'WSL_DISTRO_NAME': 'Ubuntu',
        'WAYLAND_DISPLAY': 'wayland-0',
        'LOGNAME': 'lucie_unix',
        'PULSE_SERVER': 'unix:/mnt/wslg/PulseServer',
        'WSL_INTEROP': '/run/WSL/724_interop',
        'NAME': 'DESKTOP-NMM1P6P',
        '_': '/home/lucie_unix/.pyenv/versions/3.10.12/envs/Artefact/bin/python',
        'TERM': 'xterm-color',
        'PATH': '/home/lucie_unix/.pyenv/versions/Artefact/bin:/home/lucie_unix/.pyenv/versions/3.10.12/envs/Artefact/bin:/home/lucie_unix/.vscode-server/bin/e54c774e0add60467559eb0d1e229c6452cf8447/bin/remote-cli:/home/lucie_unix/.pyenv/plugins/pyenv-virtualenv/shims:/home/lucie_unix/.pyenv/

In [149]:
## Envoyer une requête POST avec nos credentials 
CLIENT_ID=os.environ.get('CLIENT_ID')
CLIENT_SECRET=os.environ.get('CLIENT_SECRET')

data = {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
}

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

auth_response = requests.post(TOKEN_URL, headers=headers, data=data)
print(auth_response.json()) # returns a dictionary
print(auth_response.json()['access_token'])

{'access_token': 'BQBz_l7IYX58zUW-eXYT6gZ-cWmvjuhBdQtZvnSaRP5VelYcpczGWr_w_NuncTf7fO8-bKX13Q70WGUS8MK4CkjAOypT5B7i1wcM-n9nVvRe71j74klqn9qPeZGcKZTT-qu96EWiSlo', 'token_type': 'Bearer', 'expires_in': 3600}
BQBz_l7IYX58zUW-eXYT6gZ-cWmvjuhBdQtZvnSaRP5VelYcpczGWr_w_NuncTf7fO8-bKX13Q70WGUS8MK4CkjAOypT5B7i1wcM-n9nVvRe71j74klqn9qPeZGcKZTT-qu96EWiSlo


In [150]:
# RECUPERER DATA EN NOUS AUTHENTIFIANT AVEC L'ACCESS TOKEN ex. info d'un artiste spécifique
## Définir le headers avec l'access_token
## Penser à utiliser f-string ou .format() pour introduire l'access_token défini ci-dessus
access_token = auth_response.json().get('access_token') # access token à mettre en variable d'env pour pas qu'un user public réutilise
print(access_token)

BQBz_l7IYX58zUW-eXYT6gZ-cWmvjuhBdQtZvnSaRP5VelYcpczGWr_w_NuncTf7fO8-bKX13Q70WGUS8MK4CkjAOypT5B7i1wcM-n9nVvRe71j74klqn9qPeZGcKZTT-qu96EWiSlo


In [151]:
## Requête GET pour obtenir la data sur l'artiste
### Ici on utilise du python propre i.e requests.get() 

# We make a GET request and store the response 
# Add the Content-Type header set to the application/x-www-form-urlencoded value.
headers = {
    'Authorization': 'Bearer {}'.format(access_token)
}

response_artist = requests.get(BASE_URL + 'artists/4Z8W4fKeB5YxbusRsdQVPb', headers=headers) 
if response_artist.status_code == 200: 
    print(response_artist.json()) # Inspect response by printing it 
else:
    print(response_artist.status_code)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'}, 'followers': {'href': None, 'total': 11473042}, 'genres': ['art rock', 'alternative rock'], 'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb', 'id': '4Z8W4fKeB5YxbusRsdQVPb', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba03696716c9ee605006047fd', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174a03696716c9ee605006047fd', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178a03696716c9ee605006047fd', 'height': 160, 'width': 160}], 'name': 'Radiohead', 'popularity': 83, 'type': 'artist', 'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}


# 2. Get artist's nb of followers & genre

## 2.1. For one specific artist for whom we have the specific url

In [152]:
response_artist.json().get('followers')

{'href': None, 'total': 11473042}

In [153]:
followers = response_artist.json().get('followers')['total']
followers

11473042

In [154]:
name = response_artist.json().get('name')
name

'Radiohead'

In [155]:
genres = response_artist.json().get('genres')
print(genres), print(len(genres))

['art rock', 'alternative rock']
2


(None, None)

In [156]:
print(f"nb of followers for {name} = {followers}")

nb of followers for Radiohead = 11473042


## 2.2. For one specific artist for whom we have the name only (but not its spotify id)

### Retrieve Spotify ID for those artists

In [157]:
# Retrieve Spotify ID (see definition of Spotify ID: https://developer.spotify.com/documentation/web-api/concepts/spotify-uris-ids)
# Inspiration: https://stackoverflow.com/questions/41566971/how-to-get-spotify-artist-id-for-the-spotify-endpoint-url
response_artist_search = requests.get(BASE_URL + 'search?q=Daft+Punk&type=artist', headers=headers) 
if response_artist.status_code == 200: 
    print(response_artist_search.json()) # Inspect response by printing it 
else:
    print(response_artist_search.status_code)

{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=20&query=Daft%20Punk&type=artist', 'limit': 20, 'next': 'https://api.spotify.com/v1/search?offset=20&limit=20&query=Daft%20Punk&type=artist', 'offset': 0, 'previous': None, 'total': 811, 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'}, 'followers': {'href': None, 'total': 10467973}, 'genres': ['french house', 'electronic', 'electro'], 'href': 'https://api.spotify.com/v1/artists/4tZwfgrHOc3mvqYlEYSvVi', 'id': '4tZwfgrHOc3mvqYlEYSvVi', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba7bfd7835b5c1eee0c95fa6e', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174a7bfd7835b5c1eee0c95fa6e', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178a7bfd7835b5c1eee0c95fa6e', 'height': 160, 'width': 160}], 'name': 'Daft Punk', 'popularity': 80, 'type': 'artist', 'uri': 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'}, {'ext

In [158]:
# Version with name of artist defined first
artist_name= 'Daft Punk'
url_artist=artist_name.replace(' ','+')
'search?q='+url_artist+'&type=artist'

'search?q=Daft+Punk&type=artist'

In [159]:
response_artist_search = requests.get(BASE_URL + 'search?q='+url_artist+'&type=artist', headers=headers) 
if response_artist.status_code == 200: 
    print(response_artist_search.json()) # Inspect response by printing it 
else:
    print(response_artist_search.status_code)

{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=20&query=Daft%20Punk&type=artist', 'limit': 20, 'next': 'https://api.spotify.com/v1/search?offset=20&limit=20&query=Daft%20Punk&type=artist', 'offset': 0, 'previous': None, 'total': 811, 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'}, 'followers': {'href': None, 'total': 10467973}, 'genres': ['french house', 'electronic', 'electro'], 'href': 'https://api.spotify.com/v1/artists/4tZwfgrHOc3mvqYlEYSvVi', 'id': '4tZwfgrHOc3mvqYlEYSvVi', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba7bfd7835b5c1eee0c95fa6e', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174a7bfd7835b5c1eee0c95fa6e', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178a7bfd7835b5c1eee0c95fa6e', 'height': 160, 'width': 160}], 'name': 'Daft Punk', 'popularity': 80, 'type': 'artist', 'uri': 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'}, {'ext

In [160]:
# get artist's spotify id from key 'artists'
response_artist_search.json().get('artists')

{'href': 'https://api.spotify.com/v1/search?offset=0&limit=20&query=Daft%20Punk&type=artist',
 'limit': 20,
 'next': 'https://api.spotify.com/v1/search?offset=20&limit=20&query=Daft%20Punk&type=artist',
 'offset': 0,
 'previous': None,
 'total': 811,
 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'},
   'followers': {'href': None, 'total': 10467973},
   'genres': ['french house', 'electronic', 'electro'],
   'href': 'https://api.spotify.com/v1/artists/4tZwfgrHOc3mvqYlEYSvVi',
   'id': '4tZwfgrHOc3mvqYlEYSvVi',
   'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba7bfd7835b5c1eee0c95fa6e',
     'height': 640,
     'width': 640},
    {'url': 'https://i.scdn.co/image/ab67616100005174a7bfd7835b5c1eee0c95fa6e',
     'height': 320,
     'width': 320},
    {'url': 'https://i.scdn.co/image/ab6761610000f178a7bfd7835b5c1eee0c95fa6e',
     'height': 160,
     'width': 160}],
   'name': 'Daft Punk',
   'popularity': 80,
   'type': 'artist'

In [161]:
response_artist_search.json().get('artists')['items'][0]['id']

'4tZwfgrHOc3mvqYlEYSvVi'

### Use it to get nb of followers and genres

In [162]:
artist_id = response_artist_search.json().get('artists')['items'][0]['id']

response_artist_searched = requests.get(BASE_URL + 'artists/'+artist_id, headers=headers) 
if response_artist_searched.status_code == 200: 
    print(response_artist_searched.json()) # Inspect response by printing it 
else:
    print(response_artist_searched.status_code)


{'external_urls': {'spotify': 'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'}, 'followers': {'href': None, 'total': 10467973}, 'genres': ['french house', 'electronic', 'electro'], 'href': 'https://api.spotify.com/v1/artists/4tZwfgrHOc3mvqYlEYSvVi', 'id': '4tZwfgrHOc3mvqYlEYSvVi', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba7bfd7835b5c1eee0c95fa6e', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174a7bfd7835b5c1eee0c95fa6e', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178a7bfd7835b5c1eee0c95fa6e', 'height': 160, 'width': 160}], 'name': 'Daft Punk', 'popularity': 80, 'type': 'artist', 'uri': 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'}


In [163]:
followers_2 = response_artist_searched.json().get('followers')['total']
name_2 = response_artist_searched.json().get('name')
genres_2 = response_artist.json().get('genres')
print(f"nb of followers for {name_2} = {followers_2}\n {len(genres_2)} Genre(s) associated = {genres_2}")

nb of followers for Daft Punk = 10467973
 2 Genre(s) associated = ['art rock', 'alternative rock']


## 2.3. For a list of artists for whom we have got the names only

### a. First get the list of artist: for example, an extract of 2023 Most streamed songs

In [164]:
filepath = os.path.join('..','data','2023_Most_Streamed_clean.csv')
df = pd.read_csv(filepath)
df.head()

,Unnamed: 0,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,released_date,rank_streams
0,0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,...,Major,80,89,83,31,0,8,4,2023-07-14,0
1,1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,...,Major,71,61,74,7,0,10,4,2023-03-23,1
2,2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,...,Major,51,32,53,17,0,31,6,2023-06-30,2
3,3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,...,Major,55,58,72,11,0,11,15,2019-08-23,3
4,4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,...,Minor,65,23,80,14,63,11,6,2023-05-18,4


In [165]:
# We only keep the artist column and the 5 first artists
df = df[['artist(s)_name','artist_count','streams']]
df = df.iloc[:5]
df

,artist(s)_name,artist_count,streams
0,"Latto, Jung Kook",2,141381703
1,Myke Towers,1,133716286
2,Olivia Rodrigo,1,140003974
3,Taylor Swift,1,800840817
4,Bad Bunny,1,303236322


In [166]:
# We get the information artist by artist esp. the main artist
def f(row):
    if row['artist_count'] == 1:
        val = row['artist(s)_name']
    else:
        i = row['artist(s)_name'].find(',')
        val = row['artist(s)_name'][:i]
    return val

f(df.iloc[0])

'Latto'

In [167]:
def g(row):
    if row['artist_count'] == 1:
        val = ""
    else:
        i = row['artist(s)_name'].find(',')
        val = row['artist(s)_name'][i+2:]
    return val

g(df.iloc[0])

'Jung Kook'

In [168]:
df['artist_1']=df.apply(f, axis=1)
df['other artists']=df.apply(g, axis=1)
df

,artist(s)_name,artist_count,streams,artist_1,other artists
0,"Latto, Jung Kook",2,141381703,Latto,Jung Kook
1,Myke Towers,1,133716286,Myke Towers,
2,Olivia Rodrigo,1,140003974,Olivia Rodrigo,
3,Taylor Swift,1,800840817,Taylor Swift,
4,Bad Bunny,1,303236322,Bad Bunny,


### b. Now retrieve Spotify ID for those artists & get nb of followers 

In [ ]:
def fetch_followers(row):
    artist_name= row['artist_1']
    url_artist=artist_name.replace(' ','+')
    response_artist_search = requests.get(BASE_URL + 'search?q='+url_artist+'&type=artist', headers=headers) 
    
    if response_artist_search.status_code == 200: 
        artist_id = response_artist_search.json().get('artists')['items'][0]['id']
        response_artist_found = requests.get(BASE_URL + 'artists/'+artist_id, headers=headers) 
        if response_artist_found.status_code == 200: 
            followers = response_artist_found.json().get('followers')['total'] 
            return followers
        else:
            return "Error: "+ str(response_artist_found.status_code)
    else:
        return "Error: "+ str(response_artist_search.status_code)


fetch_followers(df.iloc[0])

1932096

In [170]:
df['spotify_followers_1_feb25']=df.apply(fetch_followers, axis=1)
df

,artist(s)_name,artist_count,streams,artist_1,other artists,spotify_followers_1_feb25
0,"Latto, Jung Kook",2,141381703,Latto,Jung Kook,1932096
1,Myke Towers,1,133716286,Myke Towers,,18689633
2,Olivia Rodrigo,1,140003974,Olivia Rodrigo,,43692709
3,Taylor Swift,1,800840817,Taylor Swift,,132360746
4,Bad Bunny,1,303236322,Bad Bunny,,90878445


### c. Also get genres

def fetch_followers(row):
    artist_name= row['artist_1']
    url_artist=artist_name.replace(' ','+')
    response_artist_search = requests.get(BASE_URL + 'search?q='+url_artist+'&type=artist', headers=headers) 
    
    if response_artist_search.status_code == 200: 
        artist_id = response_artist_search.json().get('artists')['items'][0]['id']
        response_artist_found = requests.get(BASE_URL + 'artists/'+artist_id, headers=headers) 
        if response_artist_found.status_code == 200: 
            followers = response_artist_found.json().get('followers')['total'] 
            genres = response_artist.json().get('genres')
            return followers
        else:
            return "Error: "+ str(response_artist_found.status_code)
    else:
        return "Error: "+ str(response_artist_search.status_code)


fetch_followers(df.iloc[0])